In [1]:
import matplotlib.pyplot as plt
from pyomo.environ import *
from pyomo.dae import *

import numpy as np
from scipy.interpolate import interp2d
import pandas as pd 
from itertools import permutations, product, combinations

from fim_doe import *
from pyomo.contrib.sensitivity_toolbox.sens import sipopt

## Define fixed bed model 

In [2]:
from fixed_bed_model_indexed_dynamic import *

alpha option:
1 / (1 + exp(-x))
    Import ComponentSet from pyomo.common.collections.  (deprecated in 5.7.1)
    (called from <frozen importlib._bootstrap>:228)


In [3]:
def add_model(doe, timepoints, time_start=0):
    '''
    add variables, equations and discretize the model time
    Argument:
        timepoints: the timesteps
        time_start: where the timesteps start. For self defined timepoints it's 0. For experimental data it's -10.
    '''
    add_variables(doe, timesteps=timepoints, start=time_start)
    
    add_equations(doe)
    
    print ('the number of timepoints is', NFEt)
    
def discretize(doe, no_points=68, collo=False):
    '''
    no_points: how many time invertals to divide. For self defined timepoints it's 69. For experimental data it's 68. 
    here it's 68
    '''
    if collo:
        TransformationFactory('dae.collocation').apply_to(doe, nfe=no_points, ncp=3, scheme='LAGRANGE-RADAU', wrt=doe.t)
    else:
        TransformationFactory('dae.finite_difference').apply_to(doe, nfe=no_points, scheme='BACKWARD', wrt=doe.t)
    
    
def initialize(doe, init_point):
    '''
    Argument: 
        init_point: initial point, csv file
    '''
    # Initialize 
    print('The init point is', init_point)    
    store = pd.read_csv(init_point)
    position_max = store['position'].max()
    store['position'] = store['position'] / position_max

    initial_bed_csv(doe, store)
    fix_initial_bed(doe)
    
def generate_exp(feed, bath, y):
    dv_dict_overall = {'temp_feed': {0: feed}, 'temp_bath': {0:bath}, 'yfeed': {0: y}}
    return dv_dict_overall

In [4]:
NFEt = 68
time_scale = 3200
collo = False
exp_time_ = time_points(NFEt, time_scale)

scena_all = {'scena-name': [0], 'fitted_transport_coefficient':212, 'ua': np.log(5.0E6)}

initialize_point = '/Users/wangjialu/dowlinglab/sorption-fixed-bed/20200729_energy.csv'
#initialize_point = './priors/20210916_feed313_bath293.csv'
#init_point = '/Users/wangjialu/dowlinglab/sorption-fixed-bed/fixed_bed_initial/T353y20_5e3.csv'
#initialize_point = './priors/20210916_feed313_bath293_5e3.csv'
#init_point = 'init_test.csv'
#init_point = 'T353y20_5e3.csv'

In [5]:
def model_integrate(scena, args_list, exp_time=exp_time_, init_point=initialize_point):
    test = create_model(scena, temp_feed=293.15, temp_bath=293.15, y=0.4, doe_model=args_list[0], k_aug=args_list[1], opt=False, diff=0)
    add_model(test, exp_time)
    discretize(test)
    initialize(test, init_point)
    return test


In [6]:
# choose from model and k_aug 
model_opt = 'doe'

if (model_opt == 'doe'):
    args_ = [True, False]
    
elif (model_opt =='kaug'):
    args_ = [True, True]
    

parameter_dict = {'fitted_transport_coefficient':212, 'ua': np.log(5.0E6)}

#dv_pass = {'temp_feed': [0], 'temp_bath':[0], 'yfeed': [0]}
#dv_pass = {'temp_feed': None, 'temp_bath':None , 'yfeed': None}
dv_pass = {'temp_feed': None, 'yfeed': None}

measurement = ['FCO2', 'temp']
measurement_extra = {'FCO2': [19], 'temp': [10, 19]}

t_measure = exp_time_
measurement_pass = {'FCO2': t_measure, 'temp': t_measure}

In [7]:
# empty prior
prior_all = np.zeros((2,2))

prior_pass=np.asarray(prior_all)

#L_initials = np.linalg.cholesky(prior_pass)
#print(L_initials)

print('The prior information FIM:', prior_pass)
print('Prior Det:', np.linalg.det(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigvals(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigh(prior_pass)[1])

The prior information FIM: [[0. 0.]
 [0. 0.]]
Prior Det: 0.0
Eigenvalue of the prior experiments FIM: [0. 0.]
Eigenvalue of the prior experiments FIM: [[1. 0.]
 [0. 1.]]


In [8]:
fim_init = [[0.0029331 , 0.01236752],
 [0.01236752 , 0.71783208]]

#f2 = open('jac_initial', 'rb')
#jac_init = pickle.load(f2)
#f2.close()

jac_init = jac_init_1.copy()
    
l_init = np.linalg.cholesky(fim_init)
print(l_init)

NameError: name 'jac_init_1' is not defined

In [ ]:
import os

scores = {} # scores is an empty dict already
target='jac_initial'
if os.path.getsize(target) > 0:      
    with open(target, "rb") as f:
        unpickler = pickle.Unpickler(f)
        # if file is not empty scores will be equal
        # to the value unpickled
        scores = unpickler.load()
        
print(scores)

## Compute FIM

In [9]:
# choose from 'simultaneous', 'sequential', 'sipopt'
#sensi_opt = 'simultaneous_finite'
sensi_opt = 'sequential_finite'
##sensi_opt = 'sequential_sipopt'
#sensi_opt = 'sequential_kaug'
#sensi_opt = 'direct_kaug'

# feed, bath, y
exp1 = generate_exp(293.15, 293.15, 0.4)

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement_pass, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_,
                                 measurement_extra_index = measurement_extra)

if_s = True 

result = doe_object.compute_FIM(exp1, mode=sensi_opt, FIM_store_name = 'fixed_bed.csv',
                               scale_nominal_param_value = if_s, 
                                store_output = 'store_output', read_output=None,
                                formula='central')

if sensi_opt == 'simultaneous_finite':
    result.extract_FIM(doe_object.m, doe_object.design_timeset,
                                                          doe_object.square_result, doe_object.obj_opt)
else: 
    result.calculate_FIM(doe_object.jac, doe_object.design_values)

The flattend measurements include: ['FCO219', 'temp10', 'temp19']
Sensitivity information is scaled by its corresponding parameter nominal value.
This scenario: {'fitted_transport_coefficient': {0: 214.12}, 'ua': {0: 15.424948470398375}, 'jac-index': {'fitted_transport_coefficient': [0], 'ua': [0]}, 'eps-abs': {'fitted_transport_coefficient': 2.1412, 'ua': 0.15424948470398375}, 'scena-name': [0]}
The inlet feed density is 41.0275814575221 [mol/m3]
the number of timepoints is 68
The init point is /Users/wangjialu/dowlinglab/sorption-fixed-bed/20200729_energy.csv
Model # of time grid is 69 , initial point # of time grid is 69
temp_feed is fixed at  293.15
yfeed is fixed at  0.4
Ipopt 3.13.3: linear_solver=ma57
halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more in

 184r 0.0000000e+00 2.77e-01 5.18e+02  -0.5 5.14e-01   1.5 1.00e+00 3.35e-01f  1
 185r 0.0000000e+00 1.30e-01 3.29e+02  -0.5 1.92e-01   1.9 1.00e+00 6.79e-01f  1
 186r 0.0000000e+00 1.26e-01 3.40e+01  -0.5 5.77e-01   1.4 1.00e+00 1.00e+00f  1
 187r 0.0000000e+00 1.24e-01 5.69e+02  -0.5 1.75e+00   1.0 1.00e+00 2.40e-01f  1
 188r 0.0000000e+00 1.20e-01 1.22e+03  -0.5 5.23e+00   0.5 1.00e+00 1.40e-01f  1
 189r 0.0000000e+00 1.17e-01 7.24e+02  -0.5 1.84e+00   0.9 1.00e+00 3.39e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 0.0000000e+00 1.75e-01 9.53e+02  -0.5 5.08e+00   0.4 9.42e-01 2.43e-01f  1
 191r 0.0000000e+00 3.34e-01 1.43e+03  -0.5 1.27e+01  -0.0 1.00e+00 1.73e-01f  1
 192r 0.0000000e+00 4.29e-01 1.11e+03  -0.5 4.34e+00   0.4 1.00e+00 3.30e-01f  1
 193r 0.0000000e+00 3.18e+00 9.20e+02  -0.5 7.16e+01    -  1.90e-01 9.31e-02f  1
 194r 0.0000000e+00 2.51e+00 1.21e+03  -0.5 7.49e+00  -0.1 1.00e+00 1.85e-01f  1
 195r 0.0000000e+00 3.74e+00

 368r 0.0000000e+00 9.47e-01 1.88e+02  -2.8 1.36e+01    -  8.53e-04 2.30e-02f  1
 369r 0.0000000e+00 9.93e-01 1.94e+02  -2.8 8.37e+00    -  9.19e-03 2.10e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 370r 0.0000000e+00 1.00e+00 1.90e+02  -2.8 8.67e+00    -  1.60e-02 1.45e-02f  1
 371r 0.0000000e+00 9.98e-01 1.59e+02  -2.8 9.42e+00    -  6.55e-02 1.51e-02f  1
 372r 0.0000000e+00 9.31e-01 1.44e+02  -2.8 9.32e-01    -  2.28e-02 6.56e-02h  1
 373r 0.0000000e+00 9.56e-01 1.44e+02  -2.8 1.16e+01    -  1.64e-02 2.29e-02f  1
 374r 0.0000000e+00 9.52e-01 1.33e+02  -2.8 1.05e+01    -  4.85e-02 2.15e-02f  1
 375r 0.0000000e+00 9.87e-01 1.38e+02  -2.8 8.62e+00    -  4.78e-02 3.00e-02f  1
 376r 0.0000000e+00 9.36e-01 1.33e+02  -2.8 7.21e-01    -  3.11e-02 5.13e-02h  1
 377r 0.0000000e+00 1.05e+00 1.23e+02  -2.8 7.11e+00    -  1.66e-02 3.79e-02f  1
 378r 0.0000000e+00 1.04e+00 1.20e+02  -2.8 5.79e+00    -  5.41e-02 3.02e-02h  1
 379r 0.0000000e+00 1.05e+00



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.3, running with linear solver ma57.

Number of nonzeros in equality constraint Jacobian...:    93745
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    51003

Total number of variables............................:    23360
                     variables with only lower bounds:        0
                variables with lower and upper bounds:    16460
                     variables with only upper bounds:        0
Total number of equality constraints.................:    23360
Total number of inequality constrai

In [ ]:
result.calculate_FIM(doe_object.jac, doe_object.design_values, jaco_involved_name=['FCO2'], jaco_involved_extra_index=[[10],[10]])




In [ ]:
print('======Result summary======')
print('Four design criteria log10() value:')
print('A-optimality:', np.log10(result.trace))
print('D-optimality:', np.log10(result.det))
print('E-optimality:', np.log10(result.min_eig))
print('Modified E-optimality:', np.log10(result.cond))

In [ ]:
print(doe_object.dsdp)

In [ ]:
import pickle

with open('store_output_botht', 'rb') as f:
    data = pickle.load(f)
    f.close()
print(data)

In [ ]:
doe_object.models[0]

In [ ]:
breakthrough_modify2(doe_object.models[2])

In [ ]:
def breakthrough_modify2(m, file=None, source="lab"):
    '''
    Draw breakthrough curve of the Pyomo model
    
    Arguments:
        m: Pyomo model
        file: when source = 'computer', this is where the computer experimental data is stored. Otherwise, its default is None
        source: if computer, it is plotting the computer experiment data. If lab, its comparing the experiments from the lab
    
    Return: None 
    
    Other: plot
    
    '''
    
    
    # unit: mol/m3
    outlet_den, _, _, _ = extract3d(m, m.C, 'CO2')
    
    # unit: cm/s
    outlet_vel, _, _, _ = extract2d(m, m.v)
    
    outlet_n2, _, _, _ = extract3d(m, m.C, 'N2')
    
    model_temp, _,_,_ = extract2d(m,m.temp)
    
    
    T = []
    for j in m.t:
        T.append(value(j))
        t_final = T[-1]
        
    
    if source == "lab":
        break_wvu = pd.read_csv('/Users/wangjialu/dowlinglab/sorption-fixed-bed/breakthrough_wvu.csv')
    
        exp = pd.read_csv('/Users/wangjialu/dowlinglab/sorption-fixed-bed/co2_breakthrough.csv')
        
        data_c1 = exp['FCO2']
        data_c = np.asarray(data_c1)
        data_t1 = exp['time']  
        data_t = np.asarray(data_t1*60+10)
        new_data = np.interp(T, data_t, data_c)
    
        fco2 = []
        for t in m.t:
            fco2.append(value(m.FCO2[0,19,t]))
        
        yco2 = []
        for i in range(len(fco2)):
            yco2.append(fco2[i]/fco2[-1])
            
        # real value of FCO2. for verification
        comp = outlet_den[-1,:]*outlet_vel[-1,:]/(outlet_den[-1, -1]*outlet_vel[-1,-1])
    
        plt.plot(exp['time']*60, exp['FCO2'],'b.', color='r', label='Experimental data')
        #plt.plot(T, outlet_den[-1,:]*outlet_vel[-1,:]/(outlet_den[-1, -1]*outlet_vel[-1,-1]), label='Model prediction')
        plt.plot(T, yco2, label='Model prediction')
        
        plt.xlabel('time [s]')
        plt.ylabel('Normalized outlet gas density of CO\N{SUBSCRIPT TWO} ')
        plt.title('Breakthrough curve')
        #plt.savefig('break_tr%.fua%.f.png'%(tr,ua))
        plt.legend()
        plt.show()
        
    elif source == "computer": 
        
        sol = pd.read_csv(file)
    
        #for i in range(len(time_exp)):    
        #    yco2_exp[time_exp[i]] = sol['FCO2'][i]
        #    temp_mid_exp[time_exp[i]] = sol['temp_mid'][i]
        #    temp_end_exp[time_exp[i]] = sol['temp_end'][i]
        
        exp_fco2 = np.asarray(sol['FCO2'].values.tolist())
        exp_temp_mid = np.asarray(sol['temp_mid'].values.tolist())
        exp_temp_end = np.asarray(sol['temp_end'].values.tolist())
        
        plt.plot(T, exp_fco2/exp_fco2[-1], label = 'Experimental data')
        plt.plot(T, outlet_den[-1,:]*outlet_vel[-1,:]/(outlet_den[-1, -1]*outlet_vel[-1,-1]), label='Model prediction')
        plt.xlabel('time [s]')
        plt.ylabel('Normalized outlet gas density of CO\N{SUBSCRIPT TWO} ')
        plt.title('Breakthrough curve')
        #plt.savefig('break_tr%.fua%.f.png'%(tr,ua))
        plt.legend()
        plt.show()
        
        
        plt.plot(T, exp_temp_mid, label='Experimental data of middle T')
        plt.plot(T, model_temp[10,:], label='Model prediction of middle T')
        plt.plot(T, exp_temp_end, label='Experimental data of end T')
        plt.plot(T, model_temp[19,:], label='Model prediction of end T')
        plt.xlabel('time [s]')
        plt.ylabel('Temperature [K]')
        plt.title('Temperature model prediction and experimental data')
        #plt.savefig('break_tr%.fua%.f.png'%(tr,ua))
        plt.legend()
        plt.show()

## Gradient-based optimization 

In [ ]:
# feed, bath, y
exp1 = generate_exp(313.15, 293.15, 0.15)



### Sequential initial solution 

In [ ]:
sensi_opt = 'sequential_finite'

doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement, t_measure, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_,
                                 measurement_extra_index = measurement_extra)

if_s = True 

result = doe_object.compute_FIM(exp1, mode=sensi_opt, FIM_store_name = 'fixed_bed.csv',
                               scale_nominal_param_value = if_s, 
                                store_output = 'store_output', read_output=None,
                                formula='central')

result.calculate_FIM(doe_object.jac, doe_object.design_values)


In [ ]:
print(doe_object.jac)

jac_init = np.zeros((len()))

for i, iname in enumerate(measurement):
    for j in parameter:
        for t in measurement_time:
            jac_init[i,j,t] = doe_object.jac[iname][]

In [ ]:
fim_init = result.FIM

#jac_init = jac_init_1.copy()
    
l_init = np.linalg.cholesky(fim_init)
print(l_init)

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement, t_measure, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_,
                                 measurement_extra_index = measurement_extra)

square_result, optimize_result = doe_object.optimize_doe(exp1, if_optimize=True, if_Cholesky=True, 
                                                         scale_nominal_param_value=True, objective_option='det', 
                                                         L_initial=l_init, jac_initial=jac_init, fim_initial=fim_init)

In [ ]:
jac_init_1 = doe_object.analysis_square.jac_extracted.copy()
print(type(jac_init_1))
print(jac_init_1[0,0,0])

print(jac_init_1)

fo = open('jac_initial', 'wb')
pickle.dump(jac_init_1, fo)
fo.close



In [ ]:
print('======Result summary======')
print('This optimization is solved with status:', optimize_result.status)
print('It gives solution:', optimize_result.solution)
print('The log10(OF) optimized is:', optimize_result.obj_value)
print('The result FIM is:', optimize_result.FIM)

t_list = []
for t in optimize_result.model.t:
    t_list.append(t)

T_list = []
for i in t_list:
    T_list.append(value(optimize_result.model.T[i]))

si=16
plt.rc('axes', titlesize=si)
plt.rc('axes', labelsize=si)
plt.rc('xtick', labelsize=si)
plt.rc('ytick', labelsize=si)
plt.rc('legend', fontsize=12)
    
plt.plot(t_list, T_list, 'b', linewidth=2)
#plt.scatter(t_list, T_list, 'b')
plt.ylabel('T [$K$]')
plt.xlabel('Time [$h$]')
plt.show()

## Heatmap

In [ ]:
#design_ranges = [list(np.linspace(293,333,5)), list(np.linspace(0.1,0.3,6))]
design_ranges = [[293, 303, 313, 323, 333], [0.1, 0.14, 0.18, 0.22, 0.26, 0.3]]

dv_apply_name = ['temp_feed','yfeed']

dv_apply_time = [[0],[0]]

# Define experiments
exp1 = generate_exp(313.15, 303.15, 0.15)
    
# choose from 'simultaneous', 'sequential', 'sipopt'
#sensi_opt = 'simultaneous_finite'
sensi_opt = 'sequential_finite'
#sensi_opt = 'sequential_sipopt'
#sensi_opt = 'sequential_kaug'
#sensi_opt = 'direct_kaug'

print(design_ranges)

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement, t_measure, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_,
                                 measurement_extra_index = measurement_extra)

all_fim = doe_object.run_grid_search(exp1, design_ranges,dv_apply_name, dv_apply_time, 
                                     mode=sensi_opt, tee_option=False,
                                    scale_nominal_param_value=True, 
                                     store_name='20210919_heatmap', read_name='20210919_heatmap'
                                    )



In [ ]:
test = all_fim.extract_criteria()

fixed = {}
all_fim.figure_drawing(fixed, ['temp_feed','yfeed'], 'Fixed bed','$feed T [K]$', 'y' )

In [ ]:
na = ['temp_feed', 'yfeed']
dd = ['temp_feed', 'yfeed']
a = [[293.0, 303.0, 313.0, 323.0, 333.0], [0.1, 0.14, 0.18, 0.22, 0.26, 0.3]]

In [ ]:
print(a[na.index('temp_feed')])